In [25]:
import baostock as bs
import pandas as pd

In [229]:
def get(stock_id):
    lg = bs.login()
    fields = "date,open,high,low,close"
    df_bs =bs.query_history_k_data(stock_id, fields, start_date='2022-03-01',frequency="D", adjustflag="2")
    
    date_list = []
    while (df_bs.error_code == '0') & df_bs.next():
        date_list.append(df_bs.get_row_data())

    result = pd.DataFrame(date_list, columns = df_bs.fields)
    result.close = result.close.astype('float64')               #收盘
    result.open = result.open.astype('float64')                 #开盘
    result.high = result.high.astype('float64')                 #最高
    result.low = result.low.astype('float64')                   #最低
    result.volume = result.volume.astype('float64')             #成交量
    result.pctChg = result.pctChg.astype('float64')             #涨跌百分比


    df = result[-100:]                                  #读取过去近半年数据（100天）
    df.reset_index(drop = True,inplace = True)
    ID = stock_id[3:]
    return df,ID

In [242]:
def identify_double_bottom(df, inter):
    """Identifies the double bottom pattern in a dataframe"""
    
 
    if not all(col in df.columns for col in ['date', 'open', 'close', 'high', 'low']):
        raise ValueError("Dataframe is missing required columns")
    
    
    bottoms = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close'])
    
    for i in range(2, len(df) - 2):
        if (df.loc[i, 'low'] < df.loc[i-2, 'low'] and
            df.loc[i, 'low'] <= df.loc[i-1, 'low'] and
            df.loc[i, 'low'] <= df.loc[i+1, 'low'] and
            df.loc[i, 'low'] < df.loc[i+2, 'low']):
            bottoms = bottoms.append(df.loc[i,:])
    
    # 找出区间内最低和次低的两个底部
    if len(bottoms) > 0:
        bottom1 = bottoms.loc[bottoms['low'].idxmin()]
        
        bottoms_no1 = bottoms.loc[(bottoms['low'] != bottom1['low'])]
        
        bottom2 = bottoms_no1.loc[bottoms_no1['low'].idxmin()]
        
        bottom1_open_close = bottom1[['open', 'close']].min()

       
        
        # 次低的低点低于最低的K线实体下沿
        is_valid = bottom2['low'] <= bottom1_open_close
        
        # 满足上面条件同时两个底部中其中一个需为最近的一个底部
        if is_valid and (bottoms.iloc[-1]['date'] == bottom1['date'] or bottoms.iloc[-1]['date'] == bottom2['date']):
            if bottom1['date'] > bottom2['date']:
                bottom_distance = len(df[df['date'] > bottom2['date']][df['date'] <bottom1['date']] )
                   
            else: 
                bottom_distance = len(df[df['date'] > bottom1['date']][df['date'] <bottom2['date']] )
                    
            
            if bottom_distance >= inter:
                return True, (bottom1['date'], bottom2['date'])
    
    
    return False, None

In [243]:
sz_id = pd.read_csv('C:/',dtype=str)
sh_id = pd.read_csv('C:/',dtype=str)

In [ ]:
SH = []
for ID in sh_id['code']:
    df = get(ID)[0]
    dis = 10
    
    if identify_double_bottom(df,dis)[0] and len(SH)<5:
        
        SH.append([ID,identify_double_bottom(df,dis)[1]])
        
    elif len(SH)==5:
        break

In [ ]:
SZ = []
for ID in sz_id['code']:
    df = get(ID)[0]
    dis = 10
    
    if identify_double_bottom(df,dis)[0] and len(SZ)<5:
        
        SZ.append([ID,identify_double_bottom(df,dis)[1]])
        
    elif len(SZ)==5:
        break

In [ ]:
SH，SZ